In [5]:
import numpy as np
import pandas as pd
import sys
from tensorflow.keras.datasets import mnist # mnist 데이터 셋
from tensorflow.keras.utils import to_categorical # 원핫인코딩
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from matplotlib import pyplot as plt # 학습과정 Loss와 acc 시각화
# quiz에서는 scale조정, train_test_split 등을 추가
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

- Red Wine 등급 예측
1. 데이터 셋 확보 및 전처리
    csv -> 결측치 처리 -> 독립변수와 타겟변수 분리 -> 독립변수 스케일조정,
    -> 타겟변수 원핫인코딩 -> 훈련셋과 테스트셋분리(train_test_split 이용 층화추출 (3등급과 8등급 데이터는 표본이 적어서 삭제하는 층화추출 ) )
2. 모델 구성(입력11, 출력6 | 출력9) layer층 4
3. 모델 학습 과정 설정
4. 모델학습(callbacks이용)
5. 모델 평가(그래프, 평가, 교차표)
6. 모델 저장 & 사용

# 1. 데이터 확보 & 전처리

| 컬럼명                  | 설명                    |
| -------------------- | --------------------- |
| fixed acidity        | 고정 산도                 |
| volatile acidity     | 휘발성 산도                |
| citric acid          | 구연산 함량                |
| residual sugar       | 잔여 당분                 |
| chlorides            | 염화물 함량                |
| free sulfur dioxide  | 자유 이산화황               |
| total sulfur dioxide | 총 이산화황                |
| density              | 밀도                    |
| pH                   | pH(산도)                |
| sulphates            | 황산염 함량                |
| alcohol              | 알코올 도수                |
| quality              | **와인 품질(0\~10점, 정수)** |


In [4]:
# 데이터 읽어오기
redwine = pd.read_csv('data/winequality-red.csv', sep=';')
redwine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64